<a href="https://colab.research.google.com/github/LuiizOt/Desenvolvimento_2025/blob/main/Aula3_LuizOtavio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SEM COMBOBOX

In [ ]:
from google.colab import drive
import geopandas as gpd

#drive mount
drive.mount('/content/drive', force_remount=True)

#load shp
bairros_cwb = gpd.read_file("/content/drive/My Drive/UFPR/DESENVOLVIMENTO/BAIRROS/DIVISA_DE_BAIRROS.shp").to_crs(epsg=31982)

#input
user_bairro = input("Digite o nome de um bairro: ").strip()

#filter
filtro = bairros_cwb[
    bairros_cwb['NOME'].str.replace('DIVISA DE BAIRROS', '').str.strip().str.lower() == user_bairro.lower()
]


# Método correto para adicionar área (evitando o warning)
filtro = filtro.assign(SHAPE_AREA=filtro.geometry.area)

# Exibir resultado
        # Calcular área em km²
area_km2 = filtro.geometry.area.iloc[0]/1000000
print("\nInformações do bairro:")
print(filtro[['NOME']].head())
print(f"\nÁrea do bairro selecionado: {area_km2:.2f}km²")

# Verifica os vizinhos (usando touches)
vizinhos = bairros_cwb[bairros_cwb.touches(filtro.geometry.iloc[0])]
vizinhos.head()


print('VIZINHOS: ')
print(vizinhos[['NOME']].head(10)) #limite de lista

Mounted at /content/drive
Digite o nome de um bairro: Cabral

Informações do bairro:
      NOME
67  CABRAL

Área do bairro selecionado: 2.03km²
VIZINHOS: 
          NOME
11   BOA VISTA
12  HUGO LANGE
55         AHÚ
69   BACACHERI
71      JUVEVÊ


COM COMBOBOX

In [ ]:
from ipywidgets import Dropdown
from IPython.display import display, clear_output
import geopandas as gpd

#dropdown
cbox = Dropdown(
    options=[(row['NOME'], row['CODIGO']) for _, row in bairros_cwb.sort_values('NOME').iterrows()],
    description='Bairro:'
)

#callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(cbox)

        codigo = cbox.value

        # Filtrar o município pelo código
        selecionado = bairros_cwb[bairros_cwb['CODIGO'] == codigo]

        # Calcular área em km²
        area_km2 = selecionado.geometry.area.iloc[0]/1000000

        print(f"\nÁrea do município selecionado: {area_km2:.2f}km²")
        # Verifica os vizinhos (usando touches)
        vizinhos = bairros_cwb[bairros_cwb.touches(selecionado.geometry.iloc[0])]
        print(f"Vizinhos:\n{vizinhos[['NOME']]}")
# 3. Associar a função ao Dropdown
cbox.observe(on_change)

# 4. Exibir a interface
display(cbox)

Dropdown(description='Bairro:', index=16, options=(('ABRANCHES', 50.0), ('AHÚ', 14.0), ('ALTO BOQUEIRÃO', 64.0…


Área do município selecionado: 2.03km²
Vizinhos:
          NOME
11   BOA VISTA
12  HUGO LANGE
55         AHÚ
69   BACACHERI
71      JUVEVÊ


SEM BAIRROS ADJACENTES

In [ ]:
from google.colab import drive
import geopandas as gpd
from shapely.geometry import Point
import folium

# Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

# Carrega o shapefile e reprojeta
bairros_cwb = gpd.read_file("/content/drive/My Drive/UFPR/DESENVOLVIMENTO/BAIRROS/DIVISA_DE_BAIRROS.shp")
bairros_cwb = bairros_cwb.to_crs(epsg=4326)


lon = float(input("Longitude: "))
lat = float(input("Latitude: "))


# Cria o ponto com as coordenadas informadas
ponto = Point(lon, lat)

# Filtra o bairro que contém o ponto
coord = bairros_cwb[bairros_cwb.geometry.contains(ponto)]

nome_bairro = coord.iloc[0]['NOME']
print(f"O ponto fica no bairro: {nome_bairro}")

# Cria o mapa centralizado no ponto informado
m = folium.Map(location=[lat, lon], zoom_start=15)

folium.GeoJson(
    coord.__geo_interface__,
    name="Coordenada",
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'black',
        'fillOpacity': 0.5,
        'weight': 2,
    }
).add_to(m)
m

Mounted at /content/drive
Longitude: -49.233805694540955
Latitude: -25.450014740560416
O ponto fica no bairro: JARDIM DAS AMÉRICAS


COM BAIRROS ADJACENTES

In [ ]:
from ipywidgets import Dropdown
from IPython.display import display, clear_output
import geopandas as gpd

#dropdown
cbox = Dropdown(
    options=[(row['NOME'], row['CODIGO']) for _, row in bairros_cwb.sort_values('NOME').iterrows()],
    description='Bairro:'
)

#callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(cbox)
        codigo = cbox.value
        selecionado = bairros_cwb[bairros_cwb['CODIGO'] == codigo]
        #vizinhos
        vizinhos = bairros_cwb[bairros_cwb.touches(selecionado.geometry.iloc[0])]

        #centroide do bairro
        centro = selecionado.geometry.iloc[0].centroid
        lat, lon = centro.y, centro.x


        # Cria o mapa centralizado no ponto informado
        m = folium.Map(location=[lat, lon], zoom_start=14)

        # Adiciona uma camada com todos os bairros em cinza (transparente)
        folium.GeoJson(
            bairros_cwb.__geo_interface__,
            name="Bairros",
            style_function=lambda feature: {
                'fillColor': 'black',
                'color': 'black',
                'fillOpacity': 0.5,
                'weight': 1,
            }
        ).add_to(m)

        folium.GeoJson(
            selecionado.__geo_interface__,
            name="Bairro informado",
            style_function=lambda feature: {
                'fillColor': 'white',
                'color': 'black',
                'fillOpacity': 0.5,
                'weight': 2,
            }
        ).add_to(m)

        folium.GeoJson(
            vizinhos.__geo_interface__,
            name="Bairros Vizinhos",
            style_function=lambda feature: {
                'fillColor': 'blue',
                'color': 'black',
                'fillOpacity': 0.5,
                'weight': 2,
            }
        ).add_to(m)
        display(m)

cbox.observe(on_change)
display(cbox)

NameError: name 'bairros_cwb' is not defined

ESCOLAS NO BAIRRO

In [ ]:
from ipywidgets import Dropdown
from IPython.display import display, clear_output
import geopandas as gpd

escolas = gpd.read_file("/content/drive/My Drive/UFPR/DESENVOLVIMENTO/ESCOLAS/ESCOLA_MUNICIPAL.shp").to_crs(epsg=4326)

#dropdown
cbox = Dropdown(
    options=[(row['NOME'], row['CODIGO']) for _, row in bairros_cwb.sort_values('NOME').iterrows()],
    description='Bairro:'
)

#callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(cbox)
        codigo = cbox.value
        selecionado = bairros_cwb[bairros_cwb['CODIGO'] == codigo]
        #centroide do bairro
        centro = selecionado.geometry.iloc[0].centroid
        lat, lon = centro.y, centro.x
        e_selec = escolas[escolas.within(selecionado.geometry.iloc[0])]

        # Cria o mapa centralizado no ponto informado
        m = folium.Map(location=[lat, lon], zoom_start=12)

        folium.GeoJson(
            selecionado.__geo_interface__,
            name="Bairro informado",
            style_function=lambda feature: {
                'fillColor': 'blue',
                'color': 'black',
                'fillOpacity': 0.5,
                'weight': 2,
            }
        ).add_to(m)
        for _, escola in e_selec.iterrows():
            coords = escola.geometry
            folium.Marker(
                location=[coords.y, coords.x]
            ).add_to(m)
        display(m)

cbox.observe(on_change)
display(cbox)

Dropdown(description='Bairro:', index=3, options=(('ABRANCHES', 50.0), ('AHÚ', 14.0), ('ALTO BOQUEIRÃO', 64.0)…